In [ ]:
import json
import requests

Refresh authentication in case the access token has expired

In [ ]:
# Retrieve refresh tokens from tokens.json
with open("../tokens.json", "r") as token_file:
    tokens = json.load(token_file)

# Refresh strava's access token
resp = requests.post("https://www.strava.com/oauth/token", data=tokens)
access_token = resp.json()['access_token']

Retrieve the athlete's most recent activities, and reconfigure the raw data so that each activity is grouped inside a dictionary based on ID.

In [ ]:
# Retrieve the most recent activities
resp = requests.get("https://www.strava.com/api/v3/athlete/activities",
                    params={"access_token": access_token})
raw_activities = resp.json()
activities = {str(a['id']):{"properties":a} for a in raw_activities}

Bundle activities with their respective time, altitude, moving, and GPS streams

In [ ]:
for act_id in activities:
    resp = requests.get(f"https://www.strava.com/api/v3/activities/{act_id}/streams",
                        params={"access_token": access_token,
                                "keys": "time,latlng,altitude,moving"})
    streams = resp.json()
    for stream in streams:
        s_type = stream["type"]
        activities[act_id][f"{s_type}_stream"] = stream["data"]

Save these bundled activities to json files for ingestion into PostgreSQL

In [ ]:
for a_id, data in activities.items():
    with open(f"../data/Activity_{a_id}.json", "w") as act_file:
        json.dump(data, act_file)